# SSNE Miniproject 6
### 318703 Tomasz Owienko
### 318718 Anna Schäfer
### Grupa piątek

In [6]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import itertools
from torchmetrics.classification import MulticlassAccuracy, MulticlassF1Score
from torch.utils.data import DataLoader, Dataset, random_split
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, RichProgressBar
import matplotlib.pyplot as plt
from torchmetrics.classification import MulticlassConfusionMatrix
import io
from PIL import Image
import torchvision
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split

In [7]:
RANDOM_SEED = 123
pl.seed_everything(RANDOM_SEED)

Seed set to 123


123

In [8]:
device = torch.device("cuda")
torch.set_float32_matmul_precision("high")

In [9]:
df_train = pd.read_csv('data/train_data.csv')
df_train

,review,rating
0,location not palace excellent hotel booke dthe...,4
1,respite definitely not place stay looking ultr...,3
2,stunning truly memorable spot right beach nusa...,4
3,solid business hotel near embassy stayed hotel...,3
4,nice place make sure lock money warning money ...,3
...,...,...
16387,great base explore new york stayed 4 nights en...,4
16388,wonderful advert paris wonderful introduction ...,4
16389,ideal relaxing holdiay rachel jay green liverp...,3
16390,"watch food, husband went resort 4 nights chris...",2


In [12]:
df_train.groupby('rating').count()

,review
rating,
0,1137
1,1434
2,1747
3,4831
4,7243


In [10]:
x_train, x_valid, y_train, y_valid = train_test_split(df_train.review, df_train.rating, test_size=0.2, random_state=42)

In [11]:
count_vectorizer = CountVectorizer()
x_vectorized = count_vectorizer.fit_transform(x_train.values)
tfidf_transformer = TfidfTransformer()
x_transformed = tfidf_transformer.fit_transform(x_vectorized)